# Census LEHD Origin Destination analysis
Goal: determine which census tracts have the highest number of jobs that people commute to from another county.

In [1]:
import pandas as pd
import numpy as np
import sys, os

In [2]:
# load origin destination data for california 2015
dirname = os.path.dirname(os.path.realpath("__file__"))
data = pd.read_csv("data/ca_od_main_JT00_2015.csv.gz", compression="gzip")

In [3]:
# load crosswalk table to join by
cxwalk = pd.read_csv("data/ca_xwalk.csv.gz", compression="gzip", encoding="ISO-8859-1", low_memory=False)

In [4]:
# sf bay area counties to filter on
cty_fips_list = [6001, 6013, 6041, 6055, 6075, 6081, 6085, 6095, 6097]

In [5]:
# filter rows in crosswalk to include only SF Bay counties and fields we need
cxwalk = cxwalk[cxwalk['cty'].isin(cty_fips_list)]
cxwalk = cxwalk[['trct', 'tabblk2010', 'cty', 'ctyname']]

In [6]:
# perform an inner join on crosswalk data and origin destination data to keep only blocks for SF Bay Area
data_w = data.merge(cxwalk, how="inner", left_on="w_geocode", right_on="tabblk2010")

In [7]:
# create seperate home and work county fips columns as strings to compare with one another
data_w['cty_h'] = data_w['h_geocode'].apply(str).str.slice(0,4)
data_w['cty_w'] = data_w['w_geocode'].apply(str).str.slice(0,4)
data_w

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate,trct,tabblk2010,cty,ctyname,cty_h,cty_w
0,60014001001007,60014003004007,1,0,1,0,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
1,60014001001007,60014027002024,1,1,0,0,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
2,60014001001007,60014037011000,1,0,1,0,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
3,60014001001007,60014042001011,1,0,1,0,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
4,60014001001007,60014042003000,1,0,1,0,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
5,60014001001007,60014042003004,1,0,1,0,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
6,60014001001007,60014045023003,1,0,0,1,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
7,60014001001007,60014046001000,1,0,0,1,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
8,60014001001007,60014046001009,1,0,0,1,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001
9,60014001001007,60014049003007,1,0,1,0,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6001,6001


In [8]:
# filter od blocks to those that have different work and home county fips
data_w_filtered = data_w[(data_w['cty_h'] != data_w['cty_w'])]

In [9]:
# inspect data
data_w_filtered

,w_geocode,h_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,createdate,trct,tabblk2010,cty,ctyname,cty_h,cty_w
25,60014001001007,60090004001000,1,0,0,1,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6009,6001
26,60014001001007,60133362021004,1,0,1,0,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6013,6001
27,60014001001007,60133382031019,1,0,0,1,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6013,6001
28,60014001001007,60133400022006,1,0,0,1,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6013,6001
29,60014001001007,60411191004005,1,0,1,0,0,0,1,0,0,1,20170919,6001400100,60014001001007,6001,"Alameda County, CA",6041,6001
42,60014001001017,60133381023005,1,1,0,0,0,1,0,0,0,1,20170919,6001400100,60014001001017,6001,"Alameda County, CA",6013,6001
43,60014001001017,60133490003009,1,0,1,0,1,0,0,0,0,1,20170919,6001400100,60014001001017,6001,"Alameda County, CA",6013,6001
44,60014001001017,60133512002008,1,0,0,1,0,0,1,0,0,1,20170919,6001400100,60014001001017,6001,"Alameda County, CA",6013,6001
45,60014001001017,60133860001018,1,1,0,0,0,0,1,0,0,1,20170919,6001400100,60014001001017,6001,"Alameda County, CA",6013,6001
46,60014001001017,60750215005001,1,0,1,0,0,0,1,0,0,1,20170919,6001400100,60014001001017,6001,"Alameda County, CA",6075,6001


In [10]:
# aggregate blocks to tracts and sum S000 column
data_agg = data_w_filtered.groupby(['trct', 'cty'], as_index=False)[['S000']].sum()

In [11]:
# sort by S000 column descending
data_agg = data_agg.sort_values('S000', ascending=False)

In [12]:
# inspect the data
data_agg

,trct,cty,S000
850,6075061500,6075,83356
674,6075011700,6075,67945
270,6001441503,6001,42289
884,6081602300,6081,24844
1175,6085505006,6085,24833
725,6075017601,6075,24001
1319,6085511705,6085,20124
726,6075017700,6075,19461
665,6075010500,6075,17851
1167,6085504602,6085,15550


In [13]:
# write processed data to csv file
outfile = os.path.join(dirname, 'data/lehd_od_tracts_high_migration.csv')
data_agg.to_csv(outfile, columns=['trct', 'cty', 'S000'], index=False, encoding="utf-8")